<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/ChatHaruhi3%E5%AF%B9%E6%8E%A5BaichuanAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入一个中文角色
- [x] 实现openai的message2response

In [1]:
!pip install -q datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


定义你的API key

In [ ]:
import os
# import httpx

os.environ["BAICHUAN_API_KEY"] = "BAICHUAN_API_KEY"    #填写控制台中获取的 APIKey 信息


如果你使用colab，运行以下代码

In [2]:
import os
from google.colab import userdata

os.environ["BAICHUAN_API_KEY"] = userdata.get("BAICHUAN_API_KEY")

载入ChatHaruhi

In [3]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 646, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 646 (delta 168), reused 196 (delta 135), pack-reused 402
Receiving objects: 100% (646/646), 3.08 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (440/440), done.
/content/Zero-Haruhi


定义openai的get_response

In [4]:
import requests

client = None

import os
import json
class Baichuan:
    def __init__(self, api_key):
        self.api_key = api_key

    def do_request(self, message, temperature = 0.3, top_p = 0.85, model = "Baichuan2-Turbo"):
        url = "https://api.baichuan-ai.com/v1/chat/completions"
        api_key = self.api_key

        data = {
            "model": model,
            "messages": message,
            "stream": False,
            "temperature": temperature,
            "top_p": top_p
        }

        json_data = json.dumps(data)

        headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer " + api_key
        }

        response = requests.post(url, data=json_data, headers=headers, timeout=60)

        # if response.status_code == 200:
        #     print("请求成功！")
        #     print("响应body:", response.text)
        #     print("请求成功，X-BC-Request-Id:", response.headers.get("X-BC-Request-Id"))
        # else:
        #     print("请求失败，状态码:", response.status_code)
        #     print("请求失败，body:", response.text)
        #     print("请求失败，X-BC-Request-Id:", response.headers.get("X-BC-Request-Id"))
        return response.text

def init_client():

    # 将client设置为全局变量
    global client

    # 将ERNIE_ACCESS_TOKEN作为参数值传递给OS
    baichuan_api_key = os.getenv("BAICHUAN_API_KEY")

    if baichuan_api_key is None:
        raise ValueError("环境变量'BAICHUAN_API_KEY'未设置，请确保已经定义了API密钥")

    client = Baichuan(baichuan_api_key)

def get_response(message, model_name = "Baichuan2-Turbo"):
    if client is None:
        init_client()

    # print(message_ua)
    response = client.do_request(message, model = model_name)
    data = json.loads(response)

    content = data['choices'][0]['message']['content']

    return content



In [5]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = "凉宫春日", llm = get_response )
chatbot.add_rag_prompt_after_persona()
response = chatbot.chat( user = "", text = "老师: 同学请自我介绍一下" )

print( response )

Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '老师: 同学请自我介绍一下', 'lid': 8}] rest_limit = 1505


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

同学：大家好，我叫张三，今年20岁，来自中国。我目前就读于XX大学，专业是计算机科学。我的兴趣爱好广泛，包括编程、阅读、旅行和音乐。我非常高兴能在这里与大家相识，期待在未来的日子里与大家共同进步，共同成长。谢谢大家！


In [8]:
print(chatbot.history)

[{'speaker': '', 'content': '老师: 同学请自我介绍一下'}, {'speaker': '{{role}}', 'content': '同学：大家好，我叫张三，今年20岁，来自中国。我目前就读于XX大学，专业是计算机科学。我的兴趣爱好广泛，包括编程、阅读、旅行和音乐。我非常高兴能在这里与大家相识，期待在未来的日子里与大家共同进步，共同成长。谢谢大家！'}]


In [9]:
response = chatbot.chat( user = "", text = "听说你初中时候谈了很多男朋友")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '听说你初中时候谈了很多男朋友', 'lid': 8}] rest_limit = 1502
我没有个人经历和情感状态所以没有谈过男朋友。我被设计用于和人类进行自然交流、解答问题、协助创作，帮助大众轻松、普惠的获得世界知识和专业服务。如果你有任何问题，可以随时向我提问。


来试验一下异步（暂无）

In [ ]:
# import json
# import asyncio
# from erniebot_agent.chat_models import ERNIEBot
# from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

# def init_aclient(model="ernie-4.0"):

#     # 将aclient设置为全局变量
#     global aclient

#     api_key = os.getenv("ERNIE_ACCESS_TOKEN")
#     if api_key is None:
#         raise ValueError("环境变量'ERNIE_ACCESS_TOKEN'未设置。请确保已经定义了API密钥。")
#     os.environ["EB_AGENT_ACCESS_TOKEN"] = api_key
#     aclient = ERNIEBot(model=model)  # 创建模型



# async def async_get_response( message, model="ernie-4.0" ):
#     if aclient is None:
#         init_aclient(model=model)

#     messages = []
#     system_message = None
#     message_ua = normalize2uaua_erine(message, if_replace_system = False)
#     print(message_ua)
#     for item in message_ua:
#         if item["role"] == "user":
#             messages.append(HumanMessage(item["content"]))
#         elif item["role"] == "system":
#             system_message = SystemMessage(item["content"])
#         else:
#             messages.append(AIMessage(item["content"]))
#     if system_message:
#         ai_message = await aclient.chat(messages=messages, temperature = 0.1)
#     else:
#         ai_message = await aclient.chat(messages=messages, system=system_message.content, temperature = 0.1)  # 调用模型chat接口，非流式返回

#     return ai_message.content

In [ ]:
chatbot.llm_async = async_get_response

In [ ]:
response = await chatbot.async_chat( user = "", text = "我是新来的同学鲁鲁，可惜我没有什么超能力")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '我是新来的同学鲁鲁，可惜我没有什么超能力', 'lid': 8}] rest_limit = 1495
[{'role': 'system', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「找出外星人、未来人以及超能力者，然后跟他们一起玩!」\n古泉:「啊，原来如此。」\n古泉:「真不愧是凉宫同学。」\n古泉:「没问题，我加入，今后还请大家多多指教。」\n古泉:「我是古泉。因为才刚转学过来，所以会有很多不懂的地方，还请您多多指教。」\n阿虚:「啊，我是……」\n春日:「他叫阿虚。」\n春日:「那个长得很可爱的是实玖瑠，另一个戴眼镜的女生是有希。」\n春日:「好了，这样我们就有五个人了。这么一来，学校就无话可说了!」\n###\n春日:「怎么都没发生学生一个接一个失踪，或者老师在形成密室的教室里遭遇到杀害这种刺激的事啊? 」\n阿虚:「你别说那种吓死人的话啦!」\n春日:「我参加过推理研究会喔。」\n阿虚:「咦!然后呢? 」\n春日:「真是笑死人了!直到目前为止，根本没半件像样的事件出现。而且，社员都只是些侦探推理小说迷而已，根本

让我们把这个放置到ChatHaruhi的默认配置中

In [ ]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response

chatbot = ChatHaruhi( role_name = "于谦", llm = get_response )

response = chatbot.chat( user = "", text = "郭德纲: 听说现在人工智能发展得快啊" )
print( response )


Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '郭德纲: 听说现在人工智能发展得快啊', 'lid': 11}] rest_limit = 1447
于谦: 是啊，现在的科技发展，真是越来越厉害了。人工智能也是个大热门啊。
